In [2]:
using SparseArrays
using Krylov
using LinearAlgebra
using Random
using Plots
Random.seed!(1234);  # determinism

In [3]:
function ls_solve_qr(A::SparseMatrixCSC{Float64,Int64}, 
                     b::Vector{Float64})
    F = qr(Matrix(A))
    Q = Matrix(F.Q)
    R = Matrix(F.R)
    return R \ (Matrix(Q)' * b)
end
ls_solve_backslash(A::SparseMatrixCSC{Float64,Int64}, 
                   b::Vector{Float64}) = 
    A \ b
ls_solve_lsqr(A::SparseMatrixCSC{Float64,Int64}, 
              b::Vector{Float64}, ϵ::Float64) =
    lsqr(A, b, etol=ϵ)[1]
ls_solve_lsmr(A::SparseMatrixCSC{Float64,Int64}, 
              b::Vector{Float64}, ϵ::Float64) =
    lsmr(A, b, etol=ϵ)[1]

function time_algs(m::Int64, n::Int64; dryrun=false)
    sparsity = 0.1;
    A = sprand(m, n, sparsity);
    b = A * randn(n) + rand(m) * 1e-2;
    
    if dryrun
        ls_solve_backslash(A, b);  ls_solve_qr(A, b)
        ls_solve_lsqr(A, b, 1e-3); ls_solve_lsmr(A, b, 1e-3)
        return
    end
    
    println("Backslash operator...")
    @time ls_solve_backslash(A, b)
    println("QR...")
    @time ls_solve_qr(A, b)

    tol = 1e-6
    println("LSQR (tol=$tol)...")
    @time ls_solve_lsqr(A, b, tol)
    println("LSMR (tol=$tol)...")
    @time ls_solve_lsmr(A, b, tol)        
end

# warm up to compile
time_algs(1000, 100, dryrun=true)

# timing
time_algs(10000, 1000);

Backslash operator...
  1.025388 seconds (3.17 k allocations: 489.203 MiB, 25.17% gc time)
QR...
  1.347981 seconds (22 allocations: 328.630 MiB, 10.22% gc time)
LSQR (tol=1.0e-6)...
  0.049479 seconds (1.48 k allocations: 2.339 MiB)
LSMR (tol=1.0e-6)...
  0.043428 seconds (1.80 k allocations: 2.785 MiB)


In [4]:
time_algs(50000, 2000);

Backslash operator...
 14.449101 seconds (6.17 k allocations: 4.733 GiB, 2.40% gc time)
QR...
 24.807017 seconds (22 allocations: 3.071 GiB, 1.16% gc time)
LSQR (tol=1.0e-6)...
  0.575386 seconds (1.26 k allocations: 8.346 MiB)
LSMR (tol=1.0e-6)...
  0.539024 seconds (1.43 k allocations: 8.596 MiB)


In [5]:
time_algs(200000, 3000);

Backslash operator...
187.946206 seconds (9.18 k allocations: 28.294 GiB, 0.50% gc time)
QR...


InterruptException: InterruptException:

In [6]:
m, n = 1000, 100
A = sprand(m, n, 0.2);
b = A * randn(n) + rand(m) * 1e-3;

lsqr_sol = lsqr(A, b, etol=1e-8)
lsmr_sol = lsmr(A, b, etol=1e-8)

([0.427297, -0.0493791, -0.603194, 0.280008, 0.0364618, 0.615627, -0.0662535, -0.28821, 0.468509, 0.696123  …  0.0277487, -1.86188, 2.03978, 0.982666, -0.354289, -0.65256, -0.753964, 1.63472, -0.602206, -0.686395], 
Simple stats
  solved: true
  inconsistent: true
  residuals:  [  6.6e+01  6.1e+01  1.9e+01 ...  9.2e-03  9.2e-03  9.2e-03 ]
  Aresiduals: [  6.7e+02  4.6e+02  1.3e+02 ...  9.8e-08  6.8e-08  2.0e-08 ]
  status: truncated forward error small enough
)

In [ ]:
plot( lsqr_sol[2].residuals,  lw=3, label="LSQR residual")
plot!(lsqr_sol[2].Aresiduals, lw=1, label="LSQR inv(A) residual")
plot!(lsmr_sol[2].residuals,  lw=3, ls=:dash, label="LSMR residual")
plot!(lsmr_sol[2].Aresiduals, lw=1, ls=:dash, label="LSMR inv(A) residual", yscale=:log10)